In [ ]:
import urllib3
import requests
import itertools
import pandas as pd
from bs4 import BeautifulSoup

urllib3.disable_warnings() # Fix the warnings!

In [ ]:
base = 'https://scholarshipdb.net/PhD-scholarships'

In [ ]:
html_text = requests.get(base, verify=False)
soup = BeautifulSoup(html_text.text, 'html.parser')

countries_all = []
fields = []
programmes = ([
    'PhD', 'Postdoctoral', 'Fellowship', 'Postgraduate', 
    'Master', 'Undergraduate', 'Coursework'
])
for link in soup.find_all('a'):
    url = link.get('href')
    if url.startswith('/scholarships-in'):
        country = url[17:-6]
        if ',' in country:
            countries_all.extend(country.split(',-'))
        else:
            countries_all.append(country)
    elif url.startswith('/scholarships?q='):
        if not url.startswith('/scholarships?q=PhD&'):
            fields.append(url[16:])

# Clean up country list
countries = []
for country in countries_all:
    if ('=' in country) or (len(country) < 2):
        break
    countries.append(country)

In [ ]:
combos = list(itertools.product(countries, programmes, fields))
print(combos)

In [ ]:
data = []
url = 'https://scholarshipdb.net/scholarships-in-{}/Program-{}?q={}'
for i, (country, programme, field) in enumerate(combos):
    html_text = requests.get(url.format(country, programme, field), verify=False)
    soup = BeautifulSoup(html_text.text, 'html.parser')
    try:
        count = int(soup.title.text.split()[0].replace(',', ''))
    except:
        count = 0
    data.append([country, programme, field, count])
    if i % 100 == 0:
        print(i)
records = pd.DataFrame(data, columns=['country', 'programme', 'field', 'count'])
records.to_csv('all-scholarships.csv', index=False)

In [ ]:
records